# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%notebook inline

In [3]:
!pip install transformers

In [4]:
import pandas as pd 
import numpy as np
import json

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig, XLMRobertaTokenizer, XLMRobertaConfig, DistilBertTokenizer, DistilBertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

torch.__version__

Using TensorFlow backend.


'1.5.1+cu101'

In [5]:
def read_json(filename) :
  with open(filename, 'r', encoding="utf8") as f:
    obj = json.load(f)
  return obj

def write_json(obj, filename) :
  with open(filename, 'w', encoding="utf8") as outfile:
    json.dump(obj, outfile)
  print ("Successfully write JSON obj to", filename)

class FineTunedModel :
  def __init__(self, model_type="mbert", version=1, fine_tuned_type="full", scenario=1) :
    ROOT_PATH = '/content/drive/My Drive/Tugas/Tugas Semester 8/Tugas Akhir/13516152 - Deborah Aprilia Josephine/';
    MODEL_PATH = 'model/scenario ' + str(scenario) + '/' 
    self.__tag_values = ['B-Merek', 'I-Merek', 'O', 'B-NamaProduk', 'I-NamaProduk', 'B-Varian', 'I-Varian', 'B-Ukuran', 'I-Ukuran', 'B-Penggunaan', 'I-Penggunaan', 'B-Tekstur', 'I-Tekstur', 'PAD']
    self.__model_type = model_type

    valid_models = ["xlmr", "mbert", "distilbert", "no-tuning-bert-labelling", "full-fine-tuning-bert-labelling"]
    valid_fine_tuned_type = ["full", "partial"]

    if model_type in valid_models and fine_tuned_type in valid_fine_tuned_type :
      MODEL_FILENAME = model_type + "_v" + str(version) + "_" + fine_tuned_type + ".pth"
    else :
      MODEL_FILENAME = "mbert_v1_full.pth"
      model_type = "mbert"

    if model_type == "xlmr":
      MODEL_NAME = 'xlm-roberta-base'
      self.__tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME, do_lower_case=False)

    elif model_type == "mbert":
      MODEL_NAME = 'bert-base-multilingual-cased'
      self.__tokenizer = BertTokenizer.from_pretrained(MODEL_NAME, do_lower_case=False)

    elif model_type == "distilbert" :
      MODEL_NAME = 'distilbert-base-multilingual-cased'
      self.__tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME, do_lower_case=False)

    else :
      MODEL_NAME = 'bert-base-multilingual-cased'
      self.__tokenizer = BertTokenizer.from_pretrained(MODEL_NAME, do_lower_case=False)
      if model_type == "no-tuning-bert-labelling" :
        MODEL_FILENAME = "model-mbert-labelling-no-tuning.pth"
      elif model_type == "full-fine-tuning-bert-labelling" :
        MODEL_FILENAME = "model-mbert-labelling-full-fine-tuning.pth"
    
    print ("Load", MODEL_PATH + MODEL_FILENAME)
    self.__model = torch.load(ROOT_PATH + MODEL_PATH + MODEL_FILENAME).cuda()

  def predict_text(self, text) :
    self.__model.eval()
    result = []
    tokenized_sentence = self.__tokenizer.encode(text)
    if (len(tokenized_sentence) > 512) :
      return None
    input_ids = torch.tensor([tokenized_sentence]).cuda()
    with torch.no_grad():
      output = self.__model(input_ids)
    label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)

    # Join Split Token
    tokens = self.__tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
    new_tokens, new_labels = [], []
    
    # Initialize Variables for XLM-R Join Token
    result_token = ""
    result_label = None
    
    for token, label_idx in zip(tokens, label_indices[0]):
      if "xlmr" in self.__model_type :
        if token.startswith("▁"):
          if result_token != "" :
            new_labels.append(result_label)
            new_tokens.append(result_token)
          result_token = token[1:]
          result_label = self.__tag_values[label_idx -1]
        else:
          result_token += token
      elif "bert" in self.__model_type :
        if token.startswith("##"):
          new_tokens[-1] = new_tokens[-1] + token[2:]
        else:
          new_labels.append(self.__tag_values[label_idx - 1])
          new_tokens.append(token)
    for token, label in zip(new_tokens, new_labels):
      elmt = {}
      elmt['token'] = token
      elmt['label'] = label
      result.append(elmt)
    # Remove [CLS] for mbert and distilbert, and <s> for xlmr
    del result[0]

    # Remove [SEP] for mbert and distilbert
    if "bert" in self.__model_type : 
      del result[-1]
    return result

  def predict(self, texts) :
    result = {}
    tokens_labels = []
    count = 0
    for idx, text in enumerate(texts) :
      result = predict_text(text)
      if result != None :
        tokens_labels.append(predict_text(text))
      else :
        count += 1
    result['tokens_labels'] = tokens_labels
    print ('Skipped text', count)
    return result

# Environment Check

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [7]:
torch.cuda.get_device_name(0)

'Tesla T4'

# Predict Demo
FineTunedModel(model_type, version, fine_tuned_type, scenario)
Example : FineTunedModel("bert", 1, "full", 1)

In [8]:
fine_tuned_model = FineTunedModel("mbert", 1, "full", 1)

Load model/scenario 1/mbert_v1_full.pth


In [9]:
fine_tuned_model.predict_text("Inez Precious Powdery Cake - Beige New Case [SEP] Buat wajahmu tampak lebih cantik dengan Inez Precious Powdery Cake-Beige 10. Hadir dengan kombinasi ekstrak mulbery, vitamin A palmitat dan E acetat, serta anti UV yang bantu lindungi kulit wajah selama beraktivitas. DETAIL Mecerahkan Kulit Wajah Hadir dengan formulasi ekstrak mulbery, Inez Precious Powdery Cake bantu cerahkan kulit wajahmu seharian. Menjaga Kelembaban Vitamin A palmitat dan E acetat yang terkandung dalam Inez Precious Powdery Cake dapat membuat kulit wajahmu tetap lembab di setiap kesempatan. Anti Oksidan Tidak hanya melembabkan kulit wajah, Inez Precious Powdery Cake mengandung vitamin A palmitat dan E acetat juga berfungsi sebagai anti oksidan bagi kulit wajahmu. Anti UV Diformulasikan dengan kandungan anti UV, Inez Precious Powdery Cake dapat melindungi wajahmu meski harus beraktivitas di luar ruangan seharian. Menutupi Noda Hitam Gak perlu takut lagi dengan noda hitam di wajahmu. Kini ada Inez Precious Powdery Cake yang dapat membantu tutupi noda hitam pada wajah dan buat penampilanmu lebih sempurna. Halal Inez Precious Powdery Cake sangat aman dan nyaman ketika digunakan karena sudah memiliki lisensi halal dari MUI Indonesia.")

[{'label': 'B-NamaProduk', 'token': 'Inez'},
 {'label': 'I-NamaProduk', 'token': 'Precious'},
 {'label': 'I-NamaProduk', 'token': 'Powdery'},
 {'label': 'I-NamaProduk', 'token': 'Cake'},
 {'label': 'I-NamaProduk', 'token': '-'},
 {'label': 'I-NamaProduk', 'token': 'Beige'},
 {'label': 'O', 'token': 'New'},
 {'label': 'I-Varian', 'token': 'Case'},
 {'label': 'O', 'token': '[SEP]'},
 {'label': 'O', 'token': 'Buat'},
 {'label': 'O', 'token': 'wajahmu'},
 {'label': 'O', 'token': 'tampak'},
 {'label': 'O', 'token': 'lebih'},
 {'label': 'O', 'token': 'cantik'},
 {'label': 'O', 'token': 'dengan'},
 {'label': 'B-NamaProduk', 'token': 'Inez'},
 {'label': 'I-NamaProduk', 'token': 'Precious'},
 {'label': 'I-NamaProduk', 'token': 'Powdery'},
 {'label': 'I-NamaProduk', 'token': 'Cake'},
 {'label': 'I-NamaProduk', 'token': '-'},
 {'label': 'B-Varian', 'token': 'Beige'},
 {'label': 'O', 'token': '10'},
 {'label': 'O', 'token': '.'},
 {'label': 'O', 'token': 'Hadir'},
 {'label': 'O', 'token': 'dengan'

In [10]:
fine_tuned_model.predict_text("Emina Double Agent Eyebrow 0.25 g [SEP] Retractable Eye brow pencil yang dilengkapi dengan sikat lembut 1. Bisa menggambar alis dengan mudah karena memiliki ujung yang bersudut 2. Ujung yang satu berupa pensil dengan formula lembut untuk membentuk alis yang terlihat alami 3. Ujung satunya berupa sikat lembut berbentuk ulir untuk hasil alis sempurna 4. Tidak perlu diserut (cukup diputar) 5. Tersedia dalam 2 varian (Brown dan Black) Agent Eyebrow Black 0.25 g NA11151202733 Agent Eyebrow Brown 0.25 g NA11171208384")

[{'label': 'B-Merek', 'token': 'Emina'},
 {'label': 'B-NamaProduk', 'token': 'Double'},
 {'label': 'I-NamaProduk', 'token': 'Agent'},
 {'label': 'I-NamaProduk', 'token': 'Eyebrow'},
 {'label': 'B-Ukuran', 'token': '0'},
 {'label': 'B-Ukuran', 'token': '.'},
 {'label': 'B-Ukuran', 'token': '25'},
 {'label': 'I-Ukuran', 'token': 'g'},
 {'label': 'O', 'token': '[SEP]'},
 {'label': 'O', 'token': 'Retractable'},
 {'label': 'O', 'token': 'Eye'},
 {'label': 'O', 'token': 'brow'},
 {'label': 'O', 'token': 'pencil'},
 {'label': 'O', 'token': 'yang'},
 {'label': 'O', 'token': 'dilengkapi'},
 {'label': 'O', 'token': 'dengan'},
 {'label': 'O', 'token': 'sikat'},
 {'label': 'O', 'token': 'lembut'},
 {'label': 'O', 'token': '1'},
 {'label': 'O', 'token': '.'},
 {'label': 'O', 'token': 'Bisa'},
 {'label': 'O', 'token': 'menggambar'},
 {'label': 'B-Penggunaan', 'token': 'alis'},
 {'label': 'O', 'token': 'dengan'},
 {'label': 'O', 'token': 'mudah'},
 {'label': 'O', 'token': 'karena'},
 {'label': 'O', '

In [11]:
fine_tuned_model = FineTunedModel("mbert", 1, "full", 2)

Load model/scenario 2/mbert_v1_full.pth


In [12]:
fine_tuned_model.predict_text("Inez Precious Powdery Cake - Beige New Case [SEP] Buat wajahmu tampak lebih cantik dengan Inez Precious Powdery Cake-Beige 10. Hadir dengan kombinasi ekstrak mulbery, vitamin A palmitat dan E acetat, serta anti UV yang bantu lindungi kulit wajah selama beraktivitas. DETAIL Mecerahkan Kulit Wajah Hadir dengan formulasi ekstrak mulbery, Inez Precious Powdery Cake bantu cerahkan kulit wajahmu seharian. Menjaga Kelembaban Vitamin A palmitat dan E acetat yang terkandung dalam Inez Precious Powdery Cake dapat membuat kulit wajahmu tetap lembab di setiap kesempatan. Anti Oksidan Tidak hanya melembabkan kulit wajah, Inez Precious Powdery Cake mengandung vitamin A palmitat dan E acetat juga berfungsi sebagai anti oksidan bagi kulit wajahmu. Anti UV Diformulasikan dengan kandungan anti UV, Inez Precious Powdery Cake dapat melindungi wajahmu meski harus beraktivitas di luar ruangan seharian. Menutupi Noda Hitam Gak perlu takut lagi dengan noda hitam di wajahmu. Kini ada Inez Precious Powdery Cake yang dapat membantu tutupi noda hitam pada wajah dan buat penampilanmu lebih sempurna. Halal Inez Precious Powdery Cake sangat aman dan nyaman ketika digunakan karena sudah memiliki lisensi halal dari MUI Indonesia.")

[{'label': 'B-NamaProduk', 'token': 'Inez'},
 {'label': 'I-NamaProduk', 'token': 'Precious'},
 {'label': 'I-NamaProduk', 'token': 'Powdery'},
 {'label': 'I-NamaProduk', 'token': 'Cake'},
 {'label': 'B-Varian', 'token': '-'},
 {'label': 'B-Varian', 'token': 'Beige'},
 {'label': 'O', 'token': 'New'},
 {'label': 'O', 'token': 'Case'},
 {'label': 'O', 'token': '[SEP]'},
 {'label': 'O', 'token': 'Buat'},
 {'label': 'O', 'token': 'wajahmu'},
 {'label': 'O', 'token': 'tampak'},
 {'label': 'O', 'token': 'lebih'},
 {'label': 'O', 'token': 'cantik'},
 {'label': 'O', 'token': 'dengan'},
 {'label': 'B-NamaProduk', 'token': 'Inez'},
 {'label': 'I-NamaProduk', 'token': 'Precious'},
 {'label': 'I-NamaProduk', 'token': 'Powdery'},
 {'label': 'I-NamaProduk', 'token': 'Cake'},
 {'label': 'B-Varian', 'token': '-'},
 {'label': 'B-Varian', 'token': 'Beige'},
 {'label': 'O', 'token': '10'},
 {'label': 'O', 'token': '.'},
 {'label': 'O', 'token': 'Hadir'},
 {'label': 'O', 'token': 'dengan'},
 {'label': 'O', 

In [13]:
fine_tuned_model.predict_text("Emina Double Agent Eyebrow 0.25 g [SEP] Retractable Eye brow pencil yang dilengkapi dengan sikat lembut 1. Bisa menggambar alis dengan mudah karena memiliki ujung yang bersudut 2. Ujung yang satu berupa pensil dengan formula lembut untuk membentuk alis yang terlihat alami 3. Ujung satunya berupa sikat lembut berbentuk ulir untuk hasil alis sempurna 4. Tidak perlu diserut (cukup diputar) 5. Tersedia dalam 2 varian (Brown dan Black) Agent Eyebrow Black 0.25 g NA11151202733 Agent Eyebrow Brown 0.25 g NA11171208384")

[{'label': 'B-Merek', 'token': 'Emina'},
 {'label': 'B-NamaProduk', 'token': 'Double'},
 {'label': 'I-NamaProduk', 'token': 'Agent'},
 {'label': 'I-NamaProduk', 'token': 'Eyebrow'},
 {'label': 'B-Ukuran', 'token': '0'},
 {'label': 'B-Ukuran', 'token': '.'},
 {'label': 'B-Ukuran', 'token': '25'},
 {'label': 'I-Ukuran', 'token': 'g'},
 {'label': 'O', 'token': '[SEP]'},
 {'label': 'O', 'token': 'Retractable'},
 {'label': 'O', 'token': 'Eye'},
 {'label': 'O', 'token': 'brow'},
 {'label': 'O', 'token': 'pencil'},
 {'label': 'O', 'token': 'yang'},
 {'label': 'O', 'token': 'dilengkapi'},
 {'label': 'O', 'token': 'dengan'},
 {'label': 'O', 'token': 'sikat'},
 {'label': 'O', 'token': 'lembut'},
 {'label': 'O', 'token': '1'},
 {'label': 'O', 'token': '.'},
 {'label': 'O', 'token': 'Bisa'},
 {'label': 'O', 'token': 'menggambar'},
 {'label': 'B-Penggunaan', 'token': 'alis'},
 {'label': 'O', 'token': 'dengan'},
 {'label': 'O', 'token': 'mudah'},
 {'label': 'O', 'token': 'karena'},
 {'label': 'O', '

In [14]:
fine_tuned_model = FineTunedModel("xlmr", 1, "full", 1)

Load model/scenario 1/xlmr_v1_full.pth


In [15]:
fine_tuned_model.predict_text("Inez Precious Powdery Cake - Beige New Case [SEP] Buat wajahmu tampak lebih cantik dengan Inez Precious Powdery Cake-Beige 10. Hadir dengan kombinasi ekstrak mulbery, vitamin A palmitat dan E acetat, serta anti UV yang bantu lindungi kulit wajah selama beraktivitas. DETAIL Mecerahkan Kulit Wajah Hadir dengan formulasi ekstrak mulbery, Inez Precious Powdery Cake bantu cerahkan kulit wajahmu seharian. Menjaga Kelembaban Vitamin A palmitat dan E acetat yang terkandung dalam Inez Precious Powdery Cake dapat membuat kulit wajahmu tetap lembab di setiap kesempatan. Anti Oksidan Tidak hanya melembabkan kulit wajah, Inez Precious Powdery Cake mengandung vitamin A palmitat dan E acetat juga berfungsi sebagai anti oksidan bagi kulit wajahmu. Anti UV Diformulasikan dengan kandungan anti UV, Inez Precious Powdery Cake dapat melindungi wajahmu meski harus beraktivitas di luar ruangan seharian. Menutupi Noda Hitam Gak perlu takut lagi dengan noda hitam di wajahmu. Kini ada Inez Precious Powdery Cake yang dapat membantu tutupi noda hitam pada wajah dan buat penampilanmu lebih sempurna. Halal Inez Precious Powdery Cake sangat aman dan nyaman ketika digunakan karena sudah memiliki lisensi halal dari MUI Indonesia.")

[{'label': 'B-NamaProduk', 'token': 'Inez'},
 {'label': 'B-NamaProduk', 'token': 'Precious'},
 {'label': 'I-NamaProduk', 'token': 'Powdery'},
 {'label': 'I-NamaProduk', 'token': 'Cake'},
 {'label': 'O', 'token': '-'},
 {'label': 'I-Varian', 'token': 'Beige'},
 {'label': 'I-Varian', 'token': 'New'},
 {'label': 'I-Varian', 'token': 'Case'},
 {'label': 'O', 'token': '[SEP]'},
 {'label': 'O', 'token': 'Buat'},
 {'label': 'O', 'token': 'wajahmu'},
 {'label': 'O', 'token': 'tampak'},
 {'label': 'O', 'token': 'lebih'},
 {'label': 'O', 'token': 'cantik'},
 {'label': 'O', 'token': 'dengan'},
 {'label': 'B-Merek', 'token': 'Inez'},
 {'label': 'B-NamaProduk', 'token': 'Precious'},
 {'label': 'I-NamaProduk', 'token': 'Powdery'},
 {'label': 'I-NamaProduk', 'token': 'Cake-Beige'},
 {'label': 'B-Ukuran', 'token': '10.'},
 {'label': 'O', 'token': 'Hadir'},
 {'label': 'O', 'token': 'dengan'},
 {'label': 'O', 'token': 'kombinasi'},
 {'label': 'O', 'token': 'ekstrak'},
 {'label': 'O', 'token': 'mulbery,'

In [16]:
fine_tuned_model.predict_text("Emina Double Agent Eyebrow 0.25 g [SEP] Retractable Eye brow pencil yang dilengkapi dengan sikat lembut 1. Bisa menggambar alis dengan mudah karena memiliki ujung yang bersudut 2. Ujung yang satu berupa pensil dengan formula lembut untuk membentuk alis yang terlihat alami 3. Ujung satunya berupa sikat lembut berbentuk ulir untuk hasil alis sempurna 4. Tidak perlu diserut (cukup diputar) 5. Tersedia dalam 2 varian (Brown dan Black) Agent Eyebrow Black 0.25 g NA11151202733 Agent Eyebrow Brown 0.25 g NA11171208384")

[{'label': 'B-Merek', 'token': 'Emina'},
 {'label': 'B-NamaProduk', 'token': 'Double'},
 {'label': 'I-NamaProduk', 'token': 'Agent'},
 {'label': 'I-NamaProduk', 'token': 'Eyebrow'},
 {'label': 'B-Ukuran', 'token': '0.25'},
 {'label': 'I-Ukuran', 'token': 'g'},
 {'label': 'O', 'token': '[SEP]'},
 {'label': 'O', 'token': 'Retractable'},
 {'label': 'O', 'token': 'Eye'},
 {'label': 'O', 'token': 'brow'},
 {'label': 'O', 'token': 'pencil'},
 {'label': 'O', 'token': 'yang'},
 {'label': 'O', 'token': 'dilengkapi'},
 {'label': 'O', 'token': 'dengan'},
 {'label': 'O', 'token': 'sikat'},
 {'label': 'O', 'token': 'lembut'},
 {'label': 'O', 'token': '1.'},
 {'label': 'O', 'token': 'Bisa'},
 {'label': 'O', 'token': 'menggambar'},
 {'label': 'B-Penggunaan', 'token': 'alis'},
 {'label': 'O', 'token': 'dengan'},
 {'label': 'O', 'token': 'mudah'},
 {'label': 'O', 'token': 'karena'},
 {'label': 'O', 'token': 'memiliki'},
 {'label': 'O', 'token': 'ujung'},
 {'label': 'O', 'token': 'yang'},
 {'label': 'O'

In [17]:
fine_tuned_model = FineTunedModel("xlmr", 1, "full", 2)

Load model/scenario 2/xlmr_v1_full.pth


In [18]:
fine_tuned_model.predict_text("Inez Precious Powdery Cake - Beige New Case [SEP] Buat wajahmu tampak lebih cantik dengan Inez Precious Powdery Cake-Beige 10. Hadir dengan kombinasi ekstrak mulbery, vitamin A palmitat dan E acetat, serta anti UV yang bantu lindungi kulit wajah selama beraktivitas. DETAIL Mecerahkan Kulit Wajah Hadir dengan formulasi ekstrak mulbery, Inez Precious Powdery Cake bantu cerahkan kulit wajahmu seharian. Menjaga Kelembaban Vitamin A palmitat dan E acetat yang terkandung dalam Inez Precious Powdery Cake dapat membuat kulit wajahmu tetap lembab di setiap kesempatan. Anti Oksidan Tidak hanya melembabkan kulit wajah, Inez Precious Powdery Cake mengandung vitamin A palmitat dan E acetat juga berfungsi sebagai anti oksidan bagi kulit wajahmu. Anti UV Diformulasikan dengan kandungan anti UV, Inez Precious Powdery Cake dapat melindungi wajahmu meski harus beraktivitas di luar ruangan seharian. Menutupi Noda Hitam Gak perlu takut lagi dengan noda hitam di wajahmu. Kini ada Inez Precious Powdery Cake yang dapat membantu tutupi noda hitam pada wajah dan buat penampilanmu lebih sempurna. Halal Inez Precious Powdery Cake sangat aman dan nyaman ketika digunakan karena sudah memiliki lisensi halal dari MUI Indonesia.")

[{'label': 'B-NamaProduk', 'token': 'Inez'},
 {'label': 'I-NamaProduk', 'token': 'Precious'},
 {'label': 'I-NamaProduk', 'token': 'Powdery'},
 {'label': 'I-NamaProduk', 'token': 'Cake'},
 {'label': 'B-Varian', 'token': '-'},
 {'label': 'B-Varian', 'token': 'Beige'},
 {'label': 'O', 'token': 'New'},
 {'label': 'I-Varian', 'token': 'Case'},
 {'label': 'O', 'token': '[SEP]'},
 {'label': 'O', 'token': 'Buat'},
 {'label': 'O', 'token': 'wajahmu'},
 {'label': 'O', 'token': 'tampak'},
 {'label': 'O', 'token': 'lebih'},
 {'label': 'O', 'token': 'cantik'},
 {'label': 'O', 'token': 'dengan'},
 {'label': 'B-NamaProduk', 'token': 'Inez'},
 {'label': 'I-NamaProduk', 'token': 'Precious'},
 {'label': 'I-NamaProduk', 'token': 'Powdery'},
 {'label': 'I-NamaProduk', 'token': 'Cake-Beige'},
 {'label': 'O', 'token': '10.'},
 {'label': 'O', 'token': 'Hadir'},
 {'label': 'O', 'token': 'dengan'},
 {'label': 'O', 'token': 'kombinasi'},
 {'label': 'O', 'token': 'ekstrak'},
 {'label': 'O', 'token': 'mulbery,'},

In [19]:
fine_tuned_model.predict_text("Emina Double Agent Eyebrow 0.25 g [SEP] Retractable Eye brow pencil yang dilengkapi dengan sikat lembut 1. Bisa menggambar alis dengan mudah karena memiliki ujung yang bersudut 2. Ujung yang satu berupa pensil dengan formula lembut untuk membentuk alis yang terlihat alami 3. Ujung satunya berupa sikat lembut berbentuk ulir untuk hasil alis sempurna 4. Tidak perlu diserut (cukup diputar) 5. Tersedia dalam 2 varian (Brown dan Black) Agent Eyebrow Black 0.25 g NA11151202733 Agent Eyebrow Brown 0.25 g NA11171208384")

[{'label': 'B-Merek', 'token': 'Emina'},
 {'label': 'B-NamaProduk', 'token': 'Double'},
 {'label': 'I-NamaProduk', 'token': 'Agent'},
 {'label': 'I-NamaProduk', 'token': 'Eyebrow'},
 {'label': 'B-Ukuran', 'token': '0.25'},
 {'label': 'I-Ukuran', 'token': 'g'},
 {'label': 'O', 'token': '[SEP]'},
 {'label': 'O', 'token': 'Retractable'},
 {'label': 'O', 'token': 'Eye'},
 {'label': 'O', 'token': 'brow'},
 {'label': 'O', 'token': 'pencil'},
 {'label': 'O', 'token': 'yang'},
 {'label': 'O', 'token': 'dilengkapi'},
 {'label': 'O', 'token': 'dengan'},
 {'label': 'O', 'token': 'sikat'},
 {'label': 'O', 'token': 'lembut'},
 {'label': 'O', 'token': '1.'},
 {'label': 'O', 'token': 'Bisa'},
 {'label': 'O', 'token': 'menggambar'},
 {'label': 'B-Penggunaan', 'token': 'alis'},
 {'label': 'O', 'token': 'dengan'},
 {'label': 'O', 'token': 'mudah'},
 {'label': 'O', 'token': 'karena'},
 {'label': 'O', 'token': 'memiliki'},
 {'label': 'O', 'token': 'ujung'},
 {'label': 'O', 'token': 'yang'},
 {'label': 'O'

# Tokopedia Automatic Labelling

In [20]:
# texts = open(ROOT_PATH + "dataset/tokopedia.txt", "r").readlines()
# print(texts[1])

In [21]:
# print (predict_text(texts[0]))

In [22]:
# obj = {}
# tokens_labels = []
# count = 0

# for idx, text in enumerate(texts) :
#   result = predict_text(text)
#   if result != None :
#     tokens_labels.append(predict_text(text))
#   else :
#     count += 1

# obj['tokens_labels'] = tokens_labels
# print ('Skipped text', count)

In [23]:
# print (len(obj['tokens_labels']))

In [24]:
# writeJSON(obj, ROOT_PATH + "dataset/tokopedia.json")